In [2]:
import numpy as np
from Data_Savior_J import load_file

Xz = load_file("./classifier_data/walk1.data")
Xz = np.vstack((Xz,load_file("./classifier_data/walk1U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk1D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk2.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk3.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk4.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk5.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5D.data")))

## Vetor de features para classificação:

$X_c = [a~| ~av~| ~aa~| ~l\_a~| ~l\_av~| ~l\_aa~| ~pos\_foot\_r~| ~pos\_foot\_l~| ~vz\_r~| ~vz\_l~| ~C]$
$X_c = [0~|~~1~~| ~~2~~| ~~~3~~| ~~~4~~~| ~~~~5~~| ~~~~~~~6~~~~~~~~~~| ~~~~~~~~7~~~~~~~~| ~~~8~~~| ~~~~9~~| 10]$

#### $a \rightarrow$ ângulo do joelho direito; $av \rightarrow$ velocidade angular do joelho direito; $aa \rightarrow$ aceleração angular do joelho direito;
#### $a \rightarrow$ ângulo do joelho esquerdo; $av \rightarrow$ velocidade angular do joelho esquerdo; $aa \rightarrow$ aceleração angular do joelho esquerdo; 
#### $pos\_foot\_r \rightarrow$ posição do tornozelo direito em relação ao sacro; $pos\_foot\_l \rightarrow$ posição do tornozelo esquerdo em relação ao sacro;
#### $vz\_r \rightarrow$ velocidade do trocanter direito no eixo z; $vz\_l \rightarrow$ velocidade do trocanter esquerdo no eixo z;
#### $C \rightarrow$ indice de classificação
## Indice de classificação $"c"$:
#### $C = 0 \rightarrow$ Marcha normal;
#### $C = 1 \rightarrow$ Marcha de subida de escada;
#### $C = 2 \rightarrow$ Marvha de descidade escada.

In [3]:
import numpy as np

X = Xz[:,[0,1,2,3,4,5,6,7,8,9]]

yz = Xz[:,[10]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((817, 10), (817,))

In [4]:
np.unique(y) # possíveis valores de y

array([ 0.,  1.,  2.])

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=10)

In [6]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


In [7]:
print X_train_std.shape

(612, 10)


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

print 'Processing time RBF'
%time rbf_svc = svm.SVC(kernel='rbf', gamma=0.2, C=6, decision_function_shape='ovr').fit(X_train_std, y_train)
print ''
print 'Processing time Polynomial'
#%time poly_svc = svm.SVC(kernel='poly', degree=4, coef0=6.0, C=0.5, decision_function_shape='ovr').fit(X_train_std, y_train)
%time poly_svc = svm.SVC(kernel='poly', degree=2, coef0=4.7, C=48.9, decision_function_shape='ovr').fit(X_train_std, y_train)

Processing time RBF
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 13.4 ms

Processing time Polynomial
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.7 ms


In [9]:
def run_svm(svc, X_test_std, y_test):
    y_pred = svc.predict(X_test_std)
    from sklearn.metrics import accuracy_score
    if (svc==rbf_svc):
        print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
    elif(svc==poly_svc):
        print ('SVM-Polynomial accuracy:--->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

In [10]:
run_svm(rbf_svc, X_test_std, y_test)
run_svm(poly_svc, X_test_std, y_test)

SVM-RBF accuracy:---------->99.02 %
SVM-Polynomial accuracy:--->99.02 %


In [157]:
from sklearn.grid_search import GridSearchCV
param_grid = [
    {
        'C'     : [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 6, 8,10, 38.5, 38.56, 38.8, 39, 39.01, 39.1, 39.2, 48.5, 48.56, 48.8, 49, 49.01, 49.1, 49.2, 50, 70, 80,
                   100, 1000], 
        'gamma' : [1000, 100, 80, 50, 35, 10, 7, 5, 3, 2, 1.5, 1, 0.9, 0.7, 0.8, 0.61, 0.62, 0.63, 0.65, 0.6, 0.59, 0.58, 0.57, 0.56, 
                   0.55, 0.5, 0.48, 0.45, 0.4, 0.35, 0.32, 0.25, 0.2, 0.1, 0.01, 0.001, 0.0001], 
        'kernel': ['rbf'],
        'random_state' : [1,5,10,20,30,40,50,60,70,80,90,100,500,1000,10000]
    },
]    
clf = GridSearchCV(svm.SVC(C=1), param_grid, cv=15)
%time clf.fit(X_train_std, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()


CPU times: user 1h 16min 7s, sys: 1.58 s, total: 1h 16min 8s
Wall time: 1h 16min 18s
Best parameters set found on development set:
()
{'kernel': 'rbf', 'C': 6, 'random_state': 1, 'gamma': 0.2}
()
Grid scores on development set:
()
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 1, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 5, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 10, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 20, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 30, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 40, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 50, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 60, 'gamma': 1000}
0.348 (+/-0.015) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 70, 'gamma': 100

In [13]:
%time
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
g = []
for i in frange(0.001, 0.015, 0.0005):
     g.append(i)
g

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


[0.001,
 0.0015,
 0.002,
 0.0025,
 0.003,
 0.0035,
 0.004,
 0.0045000000000000005,
 0.005000000000000001,
 0.005500000000000001,
 0.006000000000000002,
 0.006500000000000002,
 0.007000000000000003,
 0.007500000000000003,
 0.008000000000000004,
 0.008500000000000004,
 0.009000000000000005,
 0.009500000000000005,
 0.010000000000000005,
 0.010500000000000006,
 0.011000000000000006,
 0.011500000000000007,
 0.012000000000000007,
 0.012500000000000008,
 0.013000000000000008,
 0.013500000000000009,
 0.014000000000000009,
 0.01450000000000001]

# Gaussian Naive Bayes Classifier

In [11]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
print ('ClassifyNB accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))

ClassifyNB accuracy:---------->87.32 %


# Random Forest Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500)
rfc = rfc.fit(X_train, y_train)

pred = rfc.predict(X_test)

from sklearn.metrics import accuracy_score
print ('ClassifyNB accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))

ClassifyNB accuracy:---------->99.51 %
